### If you have Separate bash file then run it from this notebook as follows

`!sh test.sh`

#### Running your notebook needs special credentials which you can create it at the start of creating notebook instance

* AmazonSageMakerFullAccess
* AmazonEc2ContainerRegisteryFullAccess
* AmazonS3FullAccess

It is not mandatory to add amazon s3 full access you can add policy just as follows 

`{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::*"
            ]
        }
    ]
}`

### The following codes work as follows:
* creates docker image and push that to ECR
* Assigns the S3 path for training data
* Gets the role this notebook is running on
* Creates the estimator


* `Please make sure to use your ECR docker image name`
* `This code assumes docker image is running in region ap-southeast-2`

In [ ]:
#import torchnet as tnt
import torch
print(torch.__version__)

In [ ]:

%%sh

# The name of our algorithm
algorithm_name=pytorch-extending-our-containers-cat-dog-example

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-ap-southeast-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

echo fullname

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker PyTorch image
$(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = 'saeid-cat-dog'

outputpath='s3://saeid-cat-dog/output'
training_path = 's3://saeid-cat-dog/data'

role = sagemaker.get_execution_role()

In [ ]:
ecr_image = "243524247240.dkr.ecr.ap-southeast-2.amazonaws.com/pytorch-extending-our-containers-cat-dog-example:latest"
from sagemaker.estimator import Estimator
hyperparameters = {
    
                        'batch_size': 16,
                        'lr': 0.01,
                        'lr_step': 10,
                        'data_location':training_path,
                        'epochs':5,
                        'mean': [0.485, 0.456, 0.406],
                        'std':[0.229, 0.224, 0.225],
                        'no_of_classes':2,             
                        'model_def':'resnet50'
                    }


estimator = Estimator(role=role,
                      image_name=ecr_image,
                      base_job_name='docker-resnet50-cat-dog',
                      output_path=outputpath,
                      train_instance_count=1,
                      train_instance_type='ml.p3.2xlarge',
                      train_max_run=(5*24*60*60),
                      train_volume_size=20,
                      hyperparameters=hyperparameters)


In [ ]:
estimator.fit({'training': training_path})